In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.impute import KNNImputer
import requests
import os
import datetime

In [12]:
from PIL import Image

class Images:
    def __init__(self, image_dir):
        self.image_dir = image_dir

    def fetch_images(self, save_dir="images", hours=[9, 12, 15], years=range(2018, 2024)):
        """
        Télécharge les images satellites de Météociel pour les heures spécifiées de chaque jour de l'année.
        
        Args:
            save_dir (str): Dossier où sauvegarder les images. Créé si inexistant.
            hours (list): Liste des heures de la journée à télécharger (format 24h)
            years (list): Liste des années à télécharger
        
        Returns:
            int: Nombre d'images téléchargées avec succès
        """
        # Créer le répertoire de sauvegarde s'il n'existe pas
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Initialiser le compteur d'images téléchargées
        downloaded_images = 0

        # Boucle à travers chaque année
        for year in years:
            # Boucle à travers chaque jour de l'année
            for month in range(1, 13):
                for day in range(1, 32):
                    try:
                        date = datetime.date(year, month, day)
                    except ValueError:
                        continue
                    for hour in hours:
                        # Formater la date et l'heure
                        date_str = date.strftime("%d-%m-%Y")
                        print(f"Date : {date_str}")
                        hour_str = f"{hour:02d}-00"
                        print(f"Heure : {hour_str}")
                        url = f"https://neige.meteociel.fr/satellite/archives/{date_str}/satvisfr-{hour_str}.jpg"

                        # Télécharger l'image
                        response = requests.get(url)
                        if response.status_code == 200:
                            # Enregistrer l'image
                            image_path = os.path.join(save_dir, f"{date_str}_{hour_str}.jpg")
                            with open(image_path, "wb") as f:
                                f.write(response.content)
                            downloaded_images += 1
                            print(f"Téléchargé : {image_path}")
                        else:
                            print(f"Erreur de téléchargement pour {url} : {response.status_code}")
        return downloaded_images
    def crop_images(self, image_path, save_dir, coordinates:list):
        """
        Crop satellite images based on given coordinates and save them to a specified directory.
        
        Args:
            image_path (str): Path to the images folder to be cropped.
            save_dir (str): Directory where the cropped images will be saved.
            coordinates (list): List of tuples containing coordinates for cropping.
        """
        # Create the save directory if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Loop through each image in the directory
        for filename in os.listdir(image_path):
            if filename.endswith(".jpg"):
                # Open the image
                img = Image.open(os.path.join(image_path, filename))

                # Crop the image using the provided coordinates
                for i, coord in enumerate(coordinates):
                    cropped_img = img.crop(coord)
                    cropped_img.save(os.path.join(save_dir, f"{filename}"))
                    print(f"Cropped and saved: {filename}")
        return
    

    def normalize_date(self, directory):
        """
        Normalize the date format in the filenames of images in the specified directory.
        This is the normalized format: YYYY-MM-DD_HH:MM:SS.jpg
        
        Args:
            directory (str): Path to the directory containing images.
        """
        # Loop through each image in the directory
        for filename in os.listdir(directory):
            if filename.endswith(".jpg"):
                # Extract the date and time from the filename
                date_str = filename.split("_")[0]
                time_str = filename.split("_")[1].split(".")[0]

                # Convert to datetime object
                dt = datetime.datetime.strptime(date_str + " " + time_str, "%d-%m-%Y %H-%M")

                # Format the new filename
                new_filename = dt.strftime("%Y-%m-%d_%H:%M:%S") + ".jpg"

                # Rename the file
                os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
                print(f"Renamed {filename} to {new_filename}")
        return

img = Images("images")


In [11]:
img.fetch_images("images", [9, 12, 15], range(2018, 2024))
img.crop_images("images", "cropped_images", [(116, 256, 292, 388)])

Date : 01-01-2018
Heure : 09-00
Téléchargé : images/01-01-2018_09-00.jpg
Date : 01-01-2018
Heure : 12-00
Téléchargé : images/01-01-2018_12-00.jpg
Date : 01-01-2018
Heure : 15-00
Téléchargé : images/01-01-2018_15-00.jpg
Date : 02-01-2018
Heure : 09-00
Téléchargé : images/01-01-2018_12-00.jpg
Date : 01-01-2018
Heure : 15-00
Téléchargé : images/01-01-2018_15-00.jpg
Date : 02-01-2018
Heure : 09-00
Téléchargé : images/02-01-2018_09-00.jpg
Date : 02-01-2018
Heure : 12-00
Téléchargé : images/02-01-2018_12-00.jpg
Date : 02-01-2018
Heure : 15-00
Téléchargé : images/02-01-2018_09-00.jpg
Date : 02-01-2018
Heure : 12-00
Téléchargé : images/02-01-2018_12-00.jpg
Date : 02-01-2018
Heure : 15-00
Téléchargé : images/02-01-2018_15-00.jpg
Date : 03-01-2018
Heure : 09-00
Téléchargé : images/03-01-2018_09-00.jpg
Date : 03-01-2018
Heure : 12-00
Téléchargé : images/02-01-2018_15-00.jpg
Date : 03-01-2018
Heure : 09-00
Téléchargé : images/03-01-2018_09-00.jpg
Date : 03-01-2018
Heure : 12-00
Téléchargé : images

In [13]:
img.normalize_date("cropped_images")

Renamed 28-08-2020_09-00.jpg to 2020-08-28_09:00:00.jpg
Renamed 28-01-2020_15-00.jpg to 2020-01-28_15:00:00.jpg
Renamed 05-04-2021_09-00.jpg to 2021-04-05_09:00:00.jpg
Renamed 10-02-2024_15-00.jpg to 2024-02-10_15:00:00.jpg
Renamed 05-11-2021_09-00.jpg to 2021-11-05_09:00:00.jpg
Renamed 05-03-2024_12-00.jpg to 2024-03-05_12:00:00.jpg
Renamed 27-09-2022_15-00.jpg to 2022-09-27_15:00:00.jpg
Renamed 01-02-2019_09-00.jpg to 2019-02-01_09:00:00.jpg
Renamed 08-01-2025_09-00.jpg to 2025-01-08_09:00:00.jpg
Renamed 08-06-2020_12-00.jpg to 2020-06-08_12:00:00.jpg
Renamed 19-08-2018_09-00.jpg to 2018-08-19_09:00:00.jpg
Renamed 19-01-2018_15-00.jpg to 2018-01-19_15:00:00.jpg
Renamed 12-06-2022_09-00.jpg to 2022-06-12_09:00:00.jpg
Renamed 30-05-2024_12-00.jpg to 2024-05-30_12:00:00.jpg
Renamed 30-10-2024_12-00.jpg to 2024-10-30_12:00:00.jpg
Renamed 25-11-2024_15-00.jpg to 2024-11-25_15:00:00.jpg
Renamed 25-04-2024_15-00.jpg to 2024-04-25_15:00:00.jpg
Renamed 20-09-2018_12-00.jpg to 2018-09-20_12:00